In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

2023-04-03 13:53:13.581510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 13:53:13.706069: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-03 13:53:13.733091: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# Faster RCNN

In [ ]:
# TensorFlow 모델 저장소 복제
!git clone https://github.com/tensorflow/models.git

# Object Detection API 설치를 위한 패키지 설치
!apt-get install protobuf-compiler python-lxml python-pil

# TensorFlow Object Detection API 설치
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver .

# Slim 패키지 설치
!git clone https://github.com/google-research/tf-slim.git
%cd tf-slim
!python setup.py build
!python setup.py install

# 환경 변수 설정
%env PYTHONPATH $PYTHONPATH:/path/to/models/research:/path/to/models/research/slim
# 위 코드에서 /path/to/는 본인이 models 폴더를 저장한 경로로 수정

In [9]:
# Convolutional Neural Network
def cnn():
    input_layer = Input(shape=(None, None, 3))
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    return Model(inputs=input_layer, outputs=x)

# Region Proposal Network
def rpn(base_net, num_anchors):
    # 3x3 convolutional layer with 512 filters and stride of 1
    x = Conv2D(512, (3, 3), padding='same', activation='relu', name='rpn_conv')(base_net)

    # 1x1 convolutional layer with 2 * num_anchors filters for class prediction
    # The output shape is (batch_size, height, width, 2 * num_anchors)
    cls_output = Conv2D(2 * num_anchors, (1, 1), activation='softmax', name='rpn_cls')(x)

    # 1x1 convolutional layer with 4 * num_anchors filters for bbox prediction
    # The output shape is (batch_size, height, width, 4 * num_anchors)
    bbox_output = Conv2D(4 * num_anchors, (1, 1), activation='linear', name='rpn_bbox')(x)

    return [cls_output, bbox_output]
